# Simple Spotify


In [1]:
import os
import sys
sys.path.append(os.path.join(os.curdir, '..'))

In [2]:
from utils.api import OpenAPIManager

In [3]:
manager = OpenAPIManager(
    "https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml"
)

print(manager.spec_tokens)
print(manager.spec_endpoints)

175259
88


In [4]:
manager._spec.endpoints

[('GET /albums',
  'Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n',
  {'description': 'Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n',
   'parameters': [{'in': 'query',
     'name': 'ids',
     'required': True,
     'schema': {'description': 'A comma-separated list of the [Spotify IDs](/documentation/web-api/concepts/spotify-uris-ids) for the albums. Maximum: 20 IDs.\n',
      'example': '382ObEPsp2rxGrnsizN5TX,1A2GTWGtFfWp7KSQTwWOyo,2noRn2Aes5aoNVsU6iWThc',
      'title': 'Spotify Album IDs',
      'type': 'string'}}],
   'responses': {'content': {'application/json': {'schema': {'properties': {'albums': {'items': {'allOf': [{'properties': {'album_type': {'description': 'The type of the album.\n',
              'enum': ['album', 'single', 'compilation'],
              'example': 'compilation',
              'type': 'string'},
             'available_markets': {'description': 'The markets in which the alb

In [6]:
import spotipy.util as util
from langchain.requests import RequestsWrapper

from dotenv import load_dotenv

load_dotenv(os.path.join(os.curdir, "../.env"))

def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(
        raw_spec["components"]["securitySchemes"]["oauth_2_0"]["flows"][
            "authorizationCode"
        ]["scopes"].keys()
    )
    access_token = util.prompt_for_user_token(scope=",".join(scopes))
    return {"Authorization": f"Bearer {access_token}"}

# Get API credentials.
headers = construct_spotify_auth_headers(manager._raw_spec)
requests_wrapper = RequestsWrapper(headers=headers)

In [12]:
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import AzureChatOpenAI

# https://stackoverflow.com/questions/75396481/openai-gpt-3-api-error-this-models-maximum-context-length-is-4097-tokens
llm = AzureChatOpenAI(
    model='gpt-4',
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    # openai_api_version=os.getenv("OPENAI_API_VERSION"),
    # max_tokens=16385,
    # max_tokens=15000,
    # max_tokens=10000,
)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001CCB656EC10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001CCB657DB90>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://openaihzhou.openai.azure.com/', deployment_name='Streamlit', openai_api_version='2023-06-01-preview', openai_api_type='azure')

In [13]:
from langchain.agents.agent import AgentExecutor

# ValueError: You must set allow_dangerous_requests to True to use this tool. Request scan be dangerous and can lead to security vulnerabilities. For example, users can ask a server to make a request to an internalserver. It's recommended to use requests through a proxy server and avoid accepting inputs from untrusted sources without proper sandboxing.Please see: https://python.langchain.com/docs/security for further security information.
# https://github.com/langchain-ai/langchain/issues/19440
# https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.openapi.toolkit.RequestsToolkit.html
spotify_agent: AgentExecutor = planner.create_openapi_agent(
    manager._spec,
    requests_wrapper,
    llm,
    # https://github.com/langchain-ai/langchain/pull/19493
    allow_dangerous_requests=True,
    # agent_executor_kwargs=dict(handle_parsing_errors=True)
    # agent_executor_kwargs=dict(return_intermediate_steps=True, handle_parsing_errors=True),
    # agent_executor_kwargs=dict(return_intermediate_steps=True, handle_parsing_errors="Invalid output. Please call the API step by step.", max_iterations=2),
    # agent_executor_kwargs=dict(return_intermediate_steps=True, handle_parsing_errors="Invalid output. Please call the API step by step. If successfully execute the plan then end it.", max_iterations=5),
    agent_executor_kwargs=dict(
        return_intermediate_steps=True,
        # handle_parsing_errors="If successfully execute the plan then return summarize and end the plan. Otherwise, please call the API step by step.",
        handle_parsing_errors='If successfully execute the plan then return "Final Answer: `summarize`". Otherwise, please call the API step by step.',
        max_iterations=5,
        # https://github.com/langchain-ai/langchain/issues/11405
        # This is not how (I think) it work
        # trim_intermediate_steps=16384,
    ),
    # BUG: Not working
    max_context_length=16384,
)
# print(spotify_agent)
print(spotify_agent.return_intermediate_steps)
print(spotify_agent.handle_parsing_errors)
print(spotify_agent.max_iterations)
print(len(spotify_agent.tools))

True
If successfully execute the plan then return "Final Answer: `summarize`". Otherwise, please call the API step by step.
5
2


In [14]:

user_query = "give me a song I'd like, make it blues-ey (answer shortly)"
# BUG: openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 27000 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
# BUG: openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, you requested 18250 tokens (3250 in the messages, 15000 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
# BUG: openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 31472 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

# BUG: langchain_core.exceptions.OutputParserException: Could not parse LLM output: `I need to get the available genre seeds first before I can make a recommendation request with the "blues" genre.`
# handle_parsing_errors: Union[
#     bool, str, Callable[[OutputParserException], str]
# ] = False
# """How to handle errors raised by the agent's output parser.
# Defaults to `False`, which raises the error.
# If `true`, the error will be sent back to the LLM as an observation.
# If a string, the string itself will be sent to the LLM as an observation.
# If a callable function, the function will be called with the exception
#  as an argument, and the result of that function will be passed to the agent
#   as an observation.
# """
response = spotify_agent.invoke(user_query)
response




> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get a song that the user would like and is blues-ey.
Observation: 1. GET /search with the query parameter "q" set to the song name and "type" set to "track" to search for the song that the user would like.
2. GET /search with the query parameter "q" set to "genre:blues" and "type" set to "track" to search for blues-ey songs.
3. Compare the results from step 1 and step 2 to find the intersection of the two sets of songs.
4. Return the intersected songs to the user.
Thought:Parsing LLM output produced both a final answer and a parse-able action:: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /search with the query parameter "q" set to the song name and "type" set to "track" to search for the song that the user would like.
2. GET /search with the query parameter "q" set to "genre:blues" and "type" set to "track" to search for blues-ey songs.
3. 

{'input': "give me a song I'd like, make it blues-ey (answer shortly)",
 'output': 'Here are some songs that you might like and are blues-ey: [song1, song2, song3]',
 'intermediate_steps': [(AgentAction(tool='api_planner', tool_input='I need to find the right API calls to get a song that the user would like and is blues-ey.', log='Action: api_planner\nAction Input: I need to find the right API calls to get a song that the user would like and is blues-ey.'),
   '1. GET /search with the query parameter "q" set to the song name and "type" set to "track" to search for the song that the user would like.\n2. GET /search with the query parameter "q" set to "genre:blues" and "type" set to "track" to search for blues-ey songs.\n3. Compare the results from step 1 and step 2 to find the intersection of the two sets of songs.\n4. Return the intersected songs to the user.'),
  (AgentAction(tool='_Exception', tool_input='If successfully execute the plan then return "Final Answer: `summarize`". Other

In [15]:
response2 = spotify_agent.invoke("Recommend a k-pop song for me for taking shower. Keep the track url in the response")
response2



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to recommend a k-pop song for the user for taking a shower and include the track url in the response
Observation: 1. GET /recommendations/available-genre-seeds to retrieve a list of available genres for recommendations.
2. GET /search with the keyword "k-pop" to search for k-pop tracks.
3. GET /recommendations with the seed genres as "k-pop" and additional parameters for the user's shower activity (e.g., danceability, energy) to get recommended k-pop tracks for taking a shower.
4. Extract the track URL from the response and include it in the final response to the user.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /recommendations/available-genre-seeds
2. GET /search with the keyword "k-pop"
3. GET /recommendations with the seed genres as "k-pop" and additional parameters for the user's shower activity

> Entering new AgentExecutor chain

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 23804 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}